In [1]:
import os
import numpy as np 
from glob import glob
from PIL import Image
import cv2
import slideio
import json
from skimage.draw import polygon2mask
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import math
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")


In [2]:
whi_list=glob('../../data/sel_50/**/tiff/*.tiff')
tumor_list=glob('../../data/segmentation/whi/**/mask/TP_tumor/*.tiff')
normal_list=glob('../../data/segmentation/whi/**/mask/NT_normal/*.tiff')

train_tumor_path='../../data/segmentation/tile/train/TP_tumor/'
test_tumor_path='../../data/segmentation/tile/test/TP_tumor/'
train_normal_path='../../data/segmentation/tile/train/NT_normal/'
test_normal_path='../../data/segmentation/tile/test/NT_normal/'

def size_ratio(scene,img_size):
    width = scene.rect[2]
    height = scene.rect[3]
    ratio=0
    inverse_ratio=0
    img_width=0
    img_height=0
    if width>height:
        ratio=img_size/width
        inverse_ratio=width/img_size
        img_width=img_size
        img_height=height*ratio
    else:
        ratio=img_size/height
        inverse_ratio=height/img_size
        img_height=img_size
        img_width=width*ratio
        
    return int(img_width),int(img_height),inverse_ratio



In [3]:

slide_tile_size=512
for i in tqdm(range(len(whi_list))):
    tumor_count=0
    normal_count=0
    fileName=os.path.basename(os.path.splitext(whi_list[i])[0])
    slide = slideio.open_slide(whi_list[i], "GDAL")
    scene = slide.get_scene(0)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    tumor_mask=[s for s in tumor_list if fileName in s][0]
    normal_mask=[s for s in normal_list if fileName in s][0]
    if tumor_mask.find('test')==-1:
        class_group='train'
        createDirectory(train_tumor_path+fileName)
        createDirectory(train_normal_path+fileName)
    else:
        class_group='test'
        createDirectory(test_tumor_path+fileName)
        createDirectory(test_normal_path+fileName)
    tumor_image=np.array(Image.open(tumor_mask))
    normal_image=np.array(Image.open(normal_mask))
    ratio=0
    if svsWidth>svsHeight:
        ratio=svsWidth/slide_tile_size
    else:
        ratio=svsHeight/slide_tile_size
    inverse_ratio=math.floor(1/ratio*10000)/10000
    for widthCount in range(0, int(svsWidth // slide_tile_size)):
            for heightCount in range(0, int(svsHeight // slide_tile_size)):
                point_x =np.linspace(widthCount*slide_tile_size,widthCount*slide_tile_size+slide_tile_size-1,slide_tile_size,dtype=np.int32)
                point_y =np.linspace(heightCount*slide_tile_size,heightCount*slide_tile_size+slide_tile_size-1,slide_tile_size,dtype=np.int32)
                point=np.meshgrid(point_x,point_y)
                mask_point=np.copy(point)
                mask_point[0]=(mask_point[0]*inverse_ratio).astype(np.int64)
                mask_point[1]=(mask_point[1]*inverse_ratio).astype(np.int64)
                if mask_point[0].max()==tumor_image.shape[1]:
                    mask_point[0]-=1
                if mask_point[1].max()==tumor_image.shape[0]:
                    mask_point[1]-=1
                try:
                    tile_tumor_image=tumor_image[mask_point[1],mask_point[0]]/255
                    tile_normal_image=normal_image[mask_point[1],mask_point[0]]/255
                    if tile_tumor_image.mean()>0:
                        
                        image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(256,256))
                        if np.where(image[:,:,1]>=230)[0].shape[0]<(256*256)*0.8:
                            img=Image.fromarray(image)
                            tumor_count+=1
                            if class_group=='train':
                                img.save(train_tumor_path+fileName+'/'+fileName+'_'+str(tumor_count)+'.jpg')
                            else:
                                img.save(test_tumor_path+fileName+'/'+fileName+'_'+str(tumor_count)+'.jpg')
                    elif tile_normal_image.mean()>0:
                        
                        image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(256,256))
                        if np.where(image[:,:,1]>=230)[0].shape[0]<(256*256)*0.8:
                            img=Image.fromarray(image)
                            normal_count+=1
                            if class_group=='train':
                                img.save(train_normal_path+fileName+'/'+fileName+'_'+str(normal_count)+'.jpg')
                            else:
                                img.save(test_normal_path+fileName+'/'+fileName+'_'+str(normal_count)+'.jpg')
                        
                    
                except:
                    print(fileName)

  0%|          | 0/144 [00:00<?, ?it/s]